In [41]:
import tensorflow as tf
from tensorflow.keras import layers # type: ignore
from tensorflow.keras.preprocessing import image_dataset_from_directory # type: ignore
import pathlib
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score


In [42]:
batch_size = 32
img_height = 299
img_width  = 299

#Adjust the path according to your machine
data_dir_test  = pathlib.Path('data_test/')


test_ds = image_dataset_from_directory(
            data_dir_test,
            image_size=(img_height, img_width),
            batch_size=batch_size)

for image_batch, labels_batch in test_ds:
  print(f"👉The shape of each test batch is {image_batch.shape}")
  print(f"  The shape of each target batch is {labels_batch.shape}")
  break

print("👀 The classes that was used for training the model are:")
for name in test_ds.class_names:
    print(f"- {name}")

num_classes = len(test_ds.class_names)


Found 451 files belonging to 7 classes.
👉The shape of each test batch is (32, 299, 299, 3)
  The shape of each target batch is (32,)
👀 The classes that was used for training the model are:
- cataract
- degeneration
- diabets
- glaucoma
- hypertension
- myopia
- normal


In [43]:
from tensorflow.keras import layers

#Normalization of the image tensors
normalization_layer =   layers.Rescaling(1./255)
normalized_test_ds  =  test_ds.map(lambda x, y: (normalization_layer(x), y))

In [44]:

# Load the model
model = model = tf.keras.models.load_model("saved_model/Xception-01.keras")


/home/george/.pyenv/versions/3.10.6/envs/eyesense/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 9 variables whereas the saved optimizer has 1 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [45]:
y_pred = model.predict(normalized_test_ds)

15/15 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step


In [46]:
import numpy as np

y_true = np.concatenate([y.numpy() for _, y in normalized_test_ds], axis=0)
y_pred = np.argmax(y_pred, axis=1)

2025-03-21 13:52:44.545824: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [47]:
test_ds.class_names

['cataract',
 'degeneration',
 'diabets',
 'glaucoma',
 'hypertension',
 'myopia',
 'normal']

In [48]:
for class_id in range(7):
    class_recall = recall_score(
        [1 if label == class_id else 0 for label in y_true],
        [1 if pred == class_id else 0 for pred in y_pred], average = 'macro',
    )
    class_accuracy = accuracy_score(
        [1 if label == class_id else 0 for label in y_true],
        [1 if pred == class_id else 0 for pred in y_pred])

    print(f"Recall for {test_ds.class_names[class_id]}: {class_recall:.2} \t Accuracy for {test_ds.class_names[class_id]}: {class_accuracy:.2}" )

Recall for cataract: 0.47 	 Accuracy for cataract: 0.89
Recall for degeneration: 0.53 	 Accuracy for degeneration: 0.93
Recall for diabets: 0.53 	 Accuracy for diabets: 0.68
Recall for glaucoma: 0.49 	 Accuracy for glaucoma: 0.86
Recall for hypertension: 0.49 	 Accuracy for hypertension: 0.97
Recall for myopia: 0.51 	 Accuracy for myopia: 0.92
Recall for normal: 0.5 	 Accuracy for normal: 0.51


# We don't need to upload the test dataset to the clouds. We can create a dict by hand.

In [49]:
classes_dict = {0: 'cataract', 1 :'degeneration', 2:'diabets', 3: 'glaucoma', 4: 'hypertension', 5:'myopia', 6:'normal'}

In [60]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load an image and resize it to the model's expected input size (e.g., 299x299 for Xception)
image = load_img("1309_right.jpg", target_size=(299, 299))  # glaucoma image
image_array = img_to_array(image) / 255.0  # Normalize to [0, 1]
image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

In [61]:
pred = model.predict(image_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step


In [62]:
pred

array([[0.00940919, 0.02818827, 0.20571782, 0.37608874, 0.15412617,
        0.00080251, 0.22566733]], dtype=float32)

In [63]:
pred_id = np.argmax(pred, axis=1)[0]

In [64]:
classes_dict[pred_id]

'glaucoma'